# CSCI 3155: Assignment 8
## Name: Julia Troni

Goals of this assignment include: 
- Developing a type system for records and references
- A simple exercise in lazy functional programming

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (30 points)

In this problem, we will extend Lettuce language and its type system to records.  Here is a stripped down version of Lettuce we are going to use in this problem.

$$\begin{array}{rcl}
\mathbf{Expr} & \rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{Identifier})\\
& | & Plus(\mathbf{Expr}, \mathbf{Expr})\\
& | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& | & And(\mathbf{Expr}, \mathbf{Expr}) \\
& | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
& | & Let(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})\\
& | & FunDef(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}) \\
& | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
& | & \color{blue}{UnitConst} & \leftarrow \text{written as} \ () \\
& | & \color{blue}{NewRef(\mathbf{Expr})} \\
& | & \color{blue}{DeRef(\mathbf{Expr})} \\
& | & \color{blue}{AssignRef(\mathbf{Expr}, \mathbf{Expr})} & \leftarrow \text{written as} \ e_1 := e_2 \\
& | & \color{blue}{Record(List[(Identifier, \mathbf{Expr})])} & \leftarrow \text{written as} \ \{\overline{x=e}\}, \ \text{e.g.,} \ \{x_1=e_1, x_2=e_2, \ldots, x_i=e_i, \ldots \} \\
\end{array}$$

We extend the type system with a record type constructor. 
$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType & \leftarrow \text{written as } \ num \\
& | & BoolType & \leftarrow \text{written as } \ bool \\
& | & UnitType & \leftarrow \text{written as } \ unit \\
& | & FunType(\mathbf{Type} , \mathbf{Type} ) & \leftarrow \text{written as } \ t_1 \Rightarrow t_2 \\
& | & \color{red}{RefType(\mathbf{Type})} & \leftarrow \text{written as } \ ref(t_1) \\
& | & \color{red}{RecordType(List[(Identifier, \mathbf{Type})])} & \leftarrow \text{written as} \ \{\overline{x:t}\}, \ \text{e.g.,} \ \{x_1:t_1, x_2:t_2, \ldots, x_i:t_i, \ldots \} \\  
\end{array}$$




### A (10 points): Let us write some inference rules for working with records and ref types.

$$\newcommand\typeOf{\mathbf{typeOf}}$$
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$
Recall from notes on "Types and Type Checking" that $\typeOf(\texttt{e}, \alpha)$ is the type of an 
expression $\texttt{e}$ under type environment $\alpha$. The type environment maps identifiers in the current scope to  their annotated types.

Let us write a type rule for the record expression. The rule is straightforward; it simply composes the types of all sub-expressions ($e_i$) that make up the record.

$$\semRule{ \forall i.~\typeOf(e_i, \alpha) = t_i,\;\; t_i \not= \mathbf{typeerror} }
          {\typeOf(\{\overline{x=e}\}, \alpha) = \{\overline{x:t}\} }{record-ok}$$



Next, let us write a rule for NewRef.

$$\semRule{ \typeOf(\texttt{e}, \alpha) = t,\; t \not= \mathbf{typeerror} }{\typeOf(\texttt{NewRef(e)}, \alpha) = ref(t) }{newref-ok}$$

It says that if $\texttt{e}$ receives type $t$ under type environment $\alpha$ and it is not a type error, then $\texttt{NewRef(e)}$ must receive the type $ref(t)$ under $\alpha$.



(i) Complete the missing terms for the rule for `UnitConst` OK rule.
$$\semRule{}{\typeOf((), \alpha) = \color{red}{unit ???_1}}{unitconst-ok}$$

(ii) Complete the missing terms for the rule for `DeRef` OK rule.
$$\semRule{\typeOf(\texttt{e}, \alpha) = ref(t)}{\typeOf(\texttt{DeRef(e)}, \alpha) = \color{red}{t ???_2}}{deref-ok}$$

(iii) Complete the missing terms for the rule for `DeRef` error.
$$\semRule{\typeOf(\texttt{e}, \alpha) = t,\ t \not= \color{red}{ref(t) ???_3} }{\typeOf(\texttt{DeRef(e)}, \alpha) = \mathbf{typeerror}}{deref-nok}$$

(iv) Complete the missing terms for `AssignRef` OK rule.
$$\semRule{\typeOf(\texttt{e1}, \alpha) = \color{red}{ref(t) ???_4}, \typeOf(\texttt{e2}, \alpha) = t}{\typeOf(\texttt{AssignRef(e1, e2)}, \alpha) =   unit}{assignref-ok}$$


## Write your answers here

$\color{red}{???_1} $ = unit

$\color{red}{???_2} $ = t

$\color{red}{???_3} $ = ref(t)

$\color{red}{???_4} $ = ref(t)



### B (5 points)
Fill out the missing type annotations labeled type1, type2, and type3 in the program below.

``` ocaml
let z: type1 = NewRef(0) in
let f: type2 = NewRef(function (z:num) { x := z}) in 
 let g : type3 = function (z:num) { FunCall(DeRef(f),z)} in 
   let obj: type4 = {foo = f, bar=g} in
     obj
```

## Write your answers here

* type1 = RefType(NumType)
* type2 = RefType( FunType(NumType, UnitType) )
* type3 = FunType(NumType, UnitType)
* type4 = RecordType[ (foo,RefType( FunType(NumType, UnitType) ) ), (bar, FunType(NumType, UnitType) ) ]

### C (15 points): Write the type checker with references by filling in the missing code.

In [2]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type
// TODO: Write new case classes for UnitType, RefType, and RecordType
//BEGIN SOLUTION
case object UnitType extends Type
case class RefType( t1: Type) extends Type
case class RecordType(lst: List[(String, Type)] ) extends Type
//END SOLUTION

sealed trait Program
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, xType: Type, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, idType: Type, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class NewRef(e: Expr) extends Expr
case class DeRef(e: Expr) extends Expr
case class UnitConst() extends Expr
// TODO: Write new case classes for AssignRef and Record
//BEGIN SOLUTION
case class AssignRef(e1: Expr, e2: Expr) extends Expr
case class Record(lst: List[(String, Expr)] ) extends Expr
//END SOLUTION
case class TopLevel(e: Expr) extends Program

def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2
case class TypeErrorException(s: String) extends Exception


defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined object UnitType
defined class RefType
defined class RecordType
defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class UnitConst
defined class AssignRef
defined class Record
defined class TopLevel
defined function typeEquals
defined class TypeErrorException

In [3]:
def typeOf(e: Expr, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr, t1: Type, e2: Expr, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }

        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case NewRef(e) => {
            //BEGIN SOLUTION
            val t= typeOf(e, alpha)
            RefType(t)

            //END SOLUTION
        }
        
        case AssignRef(e1, e2) => {
            //BEGIN SOLUTION
            val t1= typeOf(e1, alpha)
            val t2= typeOf(e2,alpha)
            t1 match {
                case RefType(t2) => UnitType
                case _ => throw TypeErrorException(s"Expected unit type")
            }
  
            //END SOLUTION
        }
        
        case DeRef(e) => {
            //BEGIN SOLUTION
            val t= typeOf(e, alpha)
            t match {
                case RefType(r) => r
                case _ => throw TypeErrorException("Cannot DeRef something that is not a reference")
            }
            
            //END SOLUTION
        }
        
        case Record(ies) => {
            //BEGIN SOLUTION   
            
            val recT= ies.map( { 
                    case (x,e) => (x, typeOf(e,alpha))
                    case _ => throw TypeErrorException("Error")
            })
            
            RecordType(recT)
            
            //END SOLUTION
        }
        
    }
}

def typeOfProgram(p: Program) = p match {
    case TopLevel(e) => {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
            t
    }
}

defined function typeOf
defined function typeOfProgram

In [4]:
//BEGIN TEST

/* 
let x : ref(num) = NewRef(10 ) in 
   let dummy: unit = AssignRef(x, 30) in 
       DeRef(x)
       */

val p1 = Let("x", RefType(NumType), NewRef(Const(10)), Let("dummy", UnitType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t1 = typeOfProgram(TopLevel(p1))
assert(t1 == NumType, "Test 1 failed: answer should be NumType")
passed(2)
//END TEST

Program type computed successfully as NumType

*** Tests Passed (2 points) ***


p1: Let = Let(
  x = "x",
  xType = RefType(t1 = NumType),
  e1 = NewRef(e = Const(f = 10.0)),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = Ident(s = "x"), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t1: Type = NumType

In [5]:
//BEGIN TEST
/* 
let x : ref(num) = NewRef(function(z: num) z + 10) in 
   let dummy: unit = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p2 = Let("x", RefType(NumType), NewRef(fdef), Let("dummy", UnitType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t2 = try {
   typeOfProgram(TopLevel(p2))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST


*** Tests Passed (2 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p2: Let = Let(
  x = "x",
  xType = RefType(t1 = NumType),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = Ident(s = "x"), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t2: Any = "OK -- caught a type error exception: Let binding has type RefType(NumType) whereas it is bound to expression of type RefType(FunType(NumType,NumType))"

In [6]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: unit = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", UnitType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(TopLevel(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(2)
//END TEST

Program type computed successfully as FunType(NumType,NumType)

*** Tests Passed (2 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p4: Let = Let(
  x = "x",
  xType = RefType(t1 = FunType(t1 = NumType, t2 = NumType)),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = NewRef(e = Const(f = 35.0)), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t4: Type = FunType(t1 = NumType, t2 = NumType)

In [7]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p3 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t3 = try {
   typeOfProgram(TopLevel(p3))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST


*** Tests Passed (2 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p3: Let = Let(
  x = "x",
  xType = RefType(t1 = FunType(t1 = NumType, t2 = NumType)),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = NumType,
    e1 = AssignRef(e1 = Ident(s = "x"), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t3: Any = "OK -- caught a type error exception: Let binding has type NumType whereas it is bound to expression of type UnitType"

In [8]:
//BEGIN TEST

val p1 = ("foo",Const(10.0))
val t1 = ("foo",NumType)

val p2 = ("bar",Geq(Const(9.0), Const(10.0)))
val t2 = ("bar",BoolType)

val rdec1 = Record(List(p1,p2))
val rtyp1 = RecordType(List(t1,t2))

val prog1 = TopLevel(rdec1)
assert(typeOfProgram(prog1) == rtyp1, "Test failed")
passed(2)

val p3 = ("baz",NewRef(Const(11.0)))
val t3 = ("baz",RefType(NumType))
val p4 = ("qux",rdec1)
val t4 = ("qux", rtyp1)

val rdec2 = Record(List(p3,p4))
val rtyp2 = RecordType(List(t3,t4))
// prog2 = {baz=11.0, qux={foo=10.0, bar=9.0>10.0}}
val prog2 = TopLevel(rdec2)
assert(typeOfProgram(prog2) == rtyp2, "Test failed")
passed(2)


//END TEST

Program type computed successfully as RecordType(List((foo,NumType), (bar,BoolType)))

*** Tests Passed (2 points) ***
Program type computed successfully as RecordType(List((baz,RefType(NumType)), (qux,RecordType(List((foo,NumType), (bar,BoolType))))))

*** Tests Passed (2 points) ***


p1: (String, Const) = ("foo", Const(f = 10.0))
t1: (String, NumType.type) = ("foo", NumType)
p2: (String, Geq) = ("bar", Geq(e1 = Const(f = 9.0), e2 = Const(f = 10.0)))
t2: (String, BoolType.type) = ("bar", BoolType)
rdec1: Record = Record(
  lst = List(
    ("foo", Const(f = 10.0)),
    ("bar", Geq(e1 = Const(f = 9.0), e2 = Const(f = 10.0)))
  )
)
rtyp1: RecordType = RecordType(lst = List(("foo", NumType), ("bar", BoolType)))
prog1: TopLevel = TopLevel(
  e = Record(
    lst = List(
      ("foo", Const(f = 10.0)),
      ("bar", Geq(e1 = Const(f = 9.0), e2 = Const(f = 10.0)))
    )
  )
)
p3: (String, NewRef) = ("baz", NewRef(e = Const(f = 11.0)))
t3: (String, RefType) = ("baz", RefType(t1 = NumType))
p4: (String, Record) = (
  "qux",
  Record(
    lst = List(
      ("foo", Const(f = 10.0)),
      ("bar", Geq(e1 = Const(f = 9.0), e2 = Const(f = 10.0)))
    )
  )
)
t4: (String, RecordType) = (
  "qux",
  RecordType(lst = List(("foo", NumType), ("bar", BoolType)))
)
rdec2: Record = Record

In [9]:
//BEGIN TEST
/*
let x: ref(Num) = NewRef(0) in 
 let f : num => unit = function (z:num) { x := z} in 
   let obj: {set_counter:ref(num => unit)} = {set_counter = NewRef(f)} in
     obj
*/
val fdef = FunDef("z", NumType, AssignRef(Ident("x"), Ident("z")))
val gdef = FunDef("z", UnitType, DeRef(Ident("x")))
val recrd = Record(List(("set_counter",NewRef(Ident("f")))))
val rectyp = RecordType(List(("set_counter",RefType(FunType(NumType,UnitType)))))
val p3 = Let("x", RefType(NumType), NewRef(Const(0.0)), 
             Let("f", FunType(NumType, UnitType), fdef,
                   Let("obj",rectyp, recrd,
                      Ident("obj"))) )
val t = typeOfProgram(TopLevel(p3))
assert(t == rectyp, s"Test failed: answer should be $rectyp")
passed(2)

//END TEST

Program type computed successfully as RecordType(List((set_counter,RefType(FunType(NumType,UnitType)))))

*** Tests Passed (2 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = AssignRef(e1 = Ident(s = "x"), e2 = Ident(s = "z"))
)
gdef: FunDef = FunDef(
  id = "z",
  idType = UnitType,
  e = DeRef(e = Ident(s = "x"))
)
recrd: Record = Record(lst = List(("set_counter", NewRef(e = Ident(s = "f")))))
rectyp: RecordType = RecordType(
  lst = List(
    ("set_counter", RefType(t1 = FunType(t1 = NumType, t2 = UnitType)))
  )
)
p3: Let = Let(
  x = "x",
  xType = RefType(t1 = NumType),
  e1 = NewRef(e = Const(f = 0.0)),
  e2 = Let(
    x = "f",
    xType = FunType(t1 = NumType, t2 = UnitType),
    e1 = FunDef(
      id = "z",
      idType = NumType,
      e = AssignRef(e1 = Ident(s = "x"), e2 = Ident(s = "z"))
    ),
    e2 = Let(
      x = "obj",
      xType = RecordType(
        lst = List(
          ("set_counter", RefType(t1 = FunType(t1 = NumType, t2 = UnitType)))
        )
      ),
      e1 = Record(lst = List(("set_counter", NewRef(e = Ident(s = "f"))))),
      e2 = Ident(s = "obj")
    )
  )
)
t: Ty

In [10]:
//BEGIN TEST
/*
let x: ref(Num) = NewRef(0) in 
 let f : num => unit = function (z:num) { x := z} in 
  let g : unit => num = function (z:unit) {DeRef(x)} in
   let obj: {set_counter:num => unit, get_counter:unit=>num} = {set_counter = f, get_counter=g} in
     obj
*/
val fdef = FunDef("z", NumType, AssignRef(Ident("x"), Ident("z")))
val gdef = FunDef("z", UnitType, DeRef(Ident("x")))
val recrd = Record(List(("set_counter",Ident("f")), ("get_counter",Ident("g"))))
val rectyp = RecordType(List(("set_counter",FunType(NumType,UnitType)), 
                             ("get_counter",FunType(UnitType,NumType))))
val p3 = Let("x", RefType(NumType), NewRef(Const(0.0)), 
             Let("f", FunType(NumType, UnitType), fdef,
                Let("g", FunType(UnitType, NumType), gdef,
                   Let("obj",rectyp, recrd,
                      Ident("obj")))) )
val t = typeOfProgram(TopLevel(p3))
assert(t == rectyp, s"Test failed: answer should be $rectyp")
passed(3)

//END TEST

Program type computed successfully as RecordType(List((set_counter,FunType(NumType,UnitType)), (get_counter,FunType(UnitType,NumType))))

*** Tests Passed (3 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = AssignRef(e1 = Ident(s = "x"), e2 = Ident(s = "z"))
)
gdef: FunDef = FunDef(
  id = "z",
  idType = UnitType,
  e = DeRef(e = Ident(s = "x"))
)
recrd: Record = Record(
  lst = List(("set_counter", Ident(s = "f")), ("get_counter", Ident(s = "g")))
)
rectyp: RecordType = RecordType(
  lst = List(
    ("set_counter", FunType(t1 = NumType, t2 = UnitType)),
    ("get_counter", FunType(t1 = UnitType, t2 = NumType))
  )
)
p3: Let = Let(
  x = "x",
  xType = RefType(t1 = NumType),
  e1 = NewRef(e = Const(f = 0.0)),
  e2 = Let(
    x = "f",
    xType = FunType(t1 = NumType, t2 = UnitType),
    e1 = FunDef(
      id = "z",
      idType = NumType,
      e = AssignRef(e1 = Ident(s = "x"), e2 = Ident(s = "z"))
    ),
    e2 = Let(
      x = "g",
      xType = FunType(t1 = UnitType, t2 = NumType),
      e1 = FunDef(id = "z", idType = UnitType, e = DeRef(e = Ident(s = "x"))),
      e2 = Let(
        x = "obj",
        xType = RecordType(
  

## Problem 3 A ( 10 points)
Create an infinite stream of odd natural numbers starting from 1. Your stream must be a value called allOdd of type `Stream[Int]`

In [11]:
val allOdd:Stream[Int] = {
    //BEGIN SOLUTION
    def natNums(n:Int =1): Stream[Int] = { n#::(natNums(n+1)) }

    natNums().filter(x=> (x%2 ==1))

    
    //END SOLUTION
}

allOdd: Stream[Int] = Stream(
  1,
  3,
  5,
  7,
  9,
  11,
  13,
  15,
  17,
  19,
  21,
  23,
  25,
  27,
  29,
  31,
  33,
  35,
  37,
  39,
  41,
  43,
  45,
  47,
  49,
  51,
  53,
  55,
  57,
  59,
  61,
  63,
  65,
  67,
  69,
  71,
  73,
  75,
...

In [12]:
//BEGIN TEST
def testStream(n: Int): Boolean = {
    (allOdd.take(n).toList == List.range(0, 2*n).filter(_ % 2 == 1))
}
assert(testStream(5), "Testing first 5 elements of the stream")
assert(testStream(0), "Testing first 0 elements of the stream")
assert(testStream(25), "Testing first 25 elements of the stream")
assert(testStream(55), "Testing first 55 elements of the stream")
assert(testStream(1303), "Testing first 1303 elements of the stream")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function testStream

## Problem 3B (10 points)

Write an infinite stream that will generate all odd numbers that are perfect squares
  
  
Your solution should use the stream of all odd numbers created in the previous problem and a suitable application of `filter` on streams.

In [13]:
import scala.math._

val specialNatNumberStream: Stream[Int] = {
    //BEGIN SOLUTION
    allOdd.filter(x=> { 
        sqrt(x).toInt*sqrt(x).toInt == x
    } )
    
    //END SOLUTION
}

import scala.math._


specialNatNumberStream: Stream[Int] = Stream(
  1,
  9,
  25,
  49,
  81,
  121,
  169,
  225,
  289,
  361,
  441,
  529,
  625,
  729,
  841,
  961,
  1089,
  1225,
  1369,
  1521,
  1681,
  1849,
  2025,
  2209,
  2401,
  2601,
  2809,
  3025,
  3249,
  3481,
  3721,
  3969,
  4225,
  4489,
  4761,
  5041,
  5329,
  5625,
...

In [14]:
//BEGIN TEST
def cond(j: Int): Boolean = {
        val sq = math.sqrt(j).toInt
        sq*sq == j
    }

def testStream(n: Int): Boolean = {
    specialNatNumberStream.take(n).forall( cond)
}

assert(testStream(2000), "First 2000 elements test failed")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function cond
defined function testStream